In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# openai
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# huggingface
HF_TOKEN = os.getenv("HF_ACCESS_TOKEN")

# google
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_API_KEY")

# Things to try

In [ ]:
https://python.langchain.com/docs/modules/data_connection/retrievers/web_research
https://www.mlq.ai/gpt-3-enabled-research-assistant-langchain-pinecone/
https://python.langchain.com/docs/integrations/tools/wikipedia

# MultiQueryRetriever

In [ ]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load blog post
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "What are the approaches to Task Decomposition?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

# German laws to english summary

HTML2Text converts HTML into plain text (with markdown-like formatting) without any specific tag manipulation.
To extract human-readable text without needing to manipulate specific HTML elements.

In [ ]:
import openai
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer
from translate import Translator

urls = ["https://www.gesetze-im-internet.de/baf_gzuschlagsv/BJNR009350986.html",
        "https://www.gesetze-im-internet.de/baf_g/BJNR014090971.html"
        ]

loader = AsyncHtmlLoader(urls)
docs = loader.load()

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)


# Language Translation
translator = Translator(to_lang='en', from_lang='de')
german_text = docs_transformed[0].page_content  # Assuming there's one document
english_text = translator.translate(german_text[:500])

# summarization
def summarize_text(text, person_type):
    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", max_tokens=1000, temperature=0.7, top_p=0.5, frequency_penalty=0.5,
        messages=
       [ { "role": "system", "content": "You are a helpful assistant for text summarization.", },
         { "role": "user", "content": f"Summarize this for a {person_type}: {text}", }, ],
    )
    return res["choices"][0]["message"]["content"]

english_summary = summarize_text(english_text, 'person without legal training')
english_summary

# Identify Paragraph References
# You might need a parser specific to German legal texts to extract paragraph references.

# Include References in the Summary
# Incorporate the references into the English summary.

# Output
print(english_summary)

# Web retriever

In [14]:
import logging

from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma
import nest_asyncio
nest_asyncio.apply()

# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(),
    persist_directory="./chroma_db_oai",
)

# LLM
llm = ChatOpenAI( temperature=0 )

# Custom Search with Google Programmable Search Engine
search = GoogleSearchAPIWrapper()

# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    search=search,
)

# logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

# retrieve docs and provide citations
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)

user_input = "Where can I find laws regulating the duties of landlords in Berlin Germany?"
docs = web_research_retriever.get_relevant_documents(user_input)

result = qa_chain({"question": user_input})


INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'Where can I find laws regulating the duties of landlords in Berlin Germany?', 'text': LineList(lines=['1. What are the legal obligations of landlords in Berlin, Germany?\n', '2. Where can I access information on landlord responsibilities in Berlin, Germany?\n', '3. What are the laws governing landlord duties in Berlin, Germany?'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What are the legal obligations of landlords in Berlin, Germany?\n', '2. Where can I access information on landlord responsibilities in Berlin, Germany?\n', '3. What are the laws governing landlord duties in Berlin, Germany?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{

{'question': 'Where can I find laws regulating the duties of landlords in Berlin Germany?',
 'answer': 'You can find laws regulating the duties of landlords in Berlin, Germany in the German Constitution, specifically in Article 33 and Article 34. These articles outline equal citizenship and public service, as well as liability for violation of official duty. Additionally, the laws regarding property, inheritance, and expropriation are covered in Article 14. For more specific and detailed information, it is recommended to consult the relevant laws and regulations specific to Berlin. \n',
 'sources': ''}

In [17]:
result

dict_items([('question', 'Where can I find laws regulating the duties of landlords in Berlin Germany?'), ('answer', 'You can find laws regulating the duties of landlords in Berlin, Germany in the German Constitution, specifically in Article 33 and Article 34. These articles outline equal citizenship and public service, as well as liability for violation of official duty. Additionally, the laws regarding property, inheritance, and expropriation are covered in Article 14. For more specific and detailed information, it is recommended to consult the relevant laws and regulations specific to Berlin. \n'), ('sources', '')])

## WebRetriever with custom prompt and output parsing.

In [47]:
import os
import re
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers.pydantic import PydanticOutputParser

# LLMChain
search_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search 
    results. Generate FIVE Google search queries in german that are similar to
    this question. Transalte your queries into german as your searching for german official documents.
    The output should be a numbered list of questions and each
    should have a question mark at the end: {question}""",
)

class LineList(BaseModel):
    """List of questions."""

    lines: List[str] = Field(description="Questions")

class QuestionListOutputParser(PydanticOutputParser):
    """Output parser for a list of numbered questions."""

    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = re.findall(r"\d+\..*?\n", text)
        return LineList(lines=lines)
    
llm_chain = LLMChain(
            llm=llm,
            prompt=search_prompt,
            output_parser=QuestionListOutputParser(),
        )

search = GoogleSearchAPIWrapper()

# Initialize
web_research_retriever_llm_chain = WebResearchRetriever(
    vectorstore=vectorstore,
    llm_chain=llm_chain, 
    search=search, 
)


# logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

# retrieve docs and provide citations
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever, 
)


# Run
user_input = "Where can I find laws regulating the duties of landlords in Berlin Germany?"
docs = web_research_retriever_llm_chain.get_relevant_documents(user_input)
result = qa_chain({"question": user_input})

INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'Where can I find laws regulating the duties of landlords in Berlin Germany?', 'text': LineList(lines=['1. Wo kann ich Gesetze finden, die die Pflichten von Vermietern in Berlin Deutschland regeln?\n', '2. Gibt es eine offizielle Quelle, die die Rechte und Pflichten von Vermietern in Berlin Deutschland regelt?\n', '3. Wo kann ich die gesetzlichen Bestimmungen für Vermieter in Berlin Deutschland finden?\n', '4. Gibt es eine offizielle Quelle, die die Rechte und Pflichten von Vermietern in Berlin Deutschland beschreibt?\n'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. Wo kann ich Gesetze finden, die die Pflichten von Vermietern in Berlin Deutschland regeln?\n', '2. Gibt es eine offizielle Quelle, die die Rechte und Pflichten von Vermietern in Berlin Deutschland regelt?\n', '3. Wo kann ich die

{'https://www.gesetze-im-internet.de/arbschg/BJNR124610996.html'}


INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'Where can I find laws regulating the duties of landlords in Berlin Germany?', 'text': LineList(lines=['1. What are the legal obligations of landlords in Berlin, Germany?\n', '2. Where can I access information on landlord responsibilities in Berlin, Germany?\n', '3. What are the laws governing landlord duties in Berlin, Germany?'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What are the legal obligations of landlords in Berlin, Germany?\n', '2. Where can I access information on landlord responsibilities in Berlin, Germany?\n', '3. What are the laws governing landlord duties in Berlin, Germany?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': 'Basic Law for the Federal Republic of Germany', 'link': 'https://www.ges

{'question': 'Where can I find laws regulating the duties of landlords in Berlin Germany?', 'answer': ' Laws regulating the duties of landlords in Berlin, Germany can be found in the German Civil Code (Bürgerliches Gesetzbuch).\n', 'sources': 'https://www.gesetze-im-internet.de/arbschg/BJNR124610996.html'}

In [61]:
def answer(docs, result):
    sources = set()
    for d in docs:
        sources.add(d.metadata['source'])
    
    return result['answer'], sources

answer(docs, result)

(' Laws regulating the duties of landlords in Berlin, Germany can be found in the German Civil Code (Bürgerliches Gesetzbuch).\n', {'https://www.gesetze-im-internet.de/arbschg/BJNR124610996.html'})

In [ ]:
page_content='(1) This Basic Law shall require ratification by the parliaments of two thirds\nof the German _Länder_ in which it is initially to apply.\n\n(2) Insofar as the application of this Basic Law is subject to restrictions in\nany _Land_ listed in Article 23 or in any part thereof, such _Land_ or part\nthereof shall have the right to send representatives to the Bundestag in\naccordance with Article 38 and to the Bundesrat in accordance with Article 50.\n\ntable of contents\n\nArticle 145  \n[Entry into force of the Basic Law]\n\n(1) The Parliamentary Council, with the participation of the members for\nGreater Berlin, shall confirm the ratification of this Basic Law in public\nsession and shall certify and promulgate it.\n\n(2) This Basic Law shall take effect at the end of the day on which it is\npromulgated.\n\n(3) It shall be published in the Federal Law Gazette.\n\ntable of contents\n\nArticle 146  \n[Duration of the Basic Law]\n\nThis Basic Law, which, since the achievement of the unity and freedom of\nGermany, applies to the entire German people, shall cease to apply on the day\non which a constitution freely adopted by the German people takes effect.\n\ntable of contents\n\nExtracts from the German Constitution of 11 August 1919 (Weimar Constitution)  \nReligion and Religious Societies\n\ntable of contents\n\nArticle 136\n\n(1) Civil and political rights and duties shall be neither dependent upon nor\nrestricted by the exercise of religious freedom.' metadata={'source': 'https://www.gesetze-im-internet.de/englisch_gg/englisch_gg.html'}

# Google Programmable Search 
https://cse.google.com/cse.js?cx=8566b4b2cb2364df3

In [20]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper



def search(query):
    search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)
    return search.results(query, 10)


tool = Tool(
    name="Google Search Snippets",
    description="Search Google in german. Look for official german documents and laws concerning the users request.",
    func=search,
)


tool.run("What law regulates the duties of a landlord in berlin?")

[{'title': 'Basic Law for the Federal Republic of Germany',
  'link': 'https://www.gesetze-im-internet.de/englisch_gg/englisch_gg.html',
  'snippet': '(5) Details shall be regulated by federal laws. table of contents. Article 22 [Federal capital – Federal flag]. (1) Berlin is the capital of the Federal\xa0...'},
 {'title': 'Renting, Security Deposits, and Evictions - Consumer Sourcebook ...',
  'link': 'https://www.doj.nh.gov/consumer/sourcebook/renting.htm',
  'snippet': 'Leases are important legal documents governing the rights and duties of renters of any type of housing. Leases are usually prepared by landlords. The terms of\xa0...'},
 {'title': 'Landlord/Tenant Guide from the Wisconsin Department of ...',
  'link': 'https://datcp.wi.gov/Pages/Publications/LandlordTenantGuide.aspx',
  'snippet': 'State law provides a legal framework for the relationship between landlords and tenants. ... landlord-tenant rights and responsibilities in simple language. The\xa0...'},
 {'title': 'Prope

# Plan-and-execute

In [72]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [19]:
def search(query):
    search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)
    return search.results(query, 10)

llm = OpenAI(temperature=0)

tools = [
    Tool(
        name="Search",
        func=search,
        description="Find offical german documents. translate the user input into german and search for pdf, doc, docx files and provide their sources?",
    ),
]

model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

agent.run(
    "What law regulates the duties of a landlord in berlin?"
)



> Entering new PlanAndExecute chain...
steps=[Step(value='Research the legal system in Berlin to understand how landlord-tenant relationships are regulated.'), Step(value='Identify the specific law or laws that govern the duties of landlords in Berlin.'), Step(value='Review the relevant law(s) to understand the specific duties and responsibilities imposed on landlords.'), Step(value='Summarize the key points of the law(s) that regulate the duties of landlords in Berlin.'), Step(value='Given the above steps taken, provide the user with the information about the specific law that regulates the duties of landlords in Berlin.')]

> Entering new AgentExecutor chain...
Action:
{
  "action": "Search",
  "action_input": "Legal system in Berlin landlord-tenant regulations"
}

> Finished chain.
*****

Step: Research the legal system in Berlin to understand how landlord-tenant relationships are regulated.

Response: Action:
{
  "action": "Search",
  "action_input": "Legal system in Berlin landl

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4723 tokens. Please reduce the length of the messages.

# Tools


<details>
<summary>Different Agent Types:</summary>


These `ReAct` agent types serve different purposes within a system designed for research, information retrieval, and interaction. Here's an explanation of each:

1. **`AgentType.ZERO_SHOT_REACT_DESCRIPTION`: ZeroShotAgent**
   - This agent is designed for generating descriptions using a zero-shot approach. It can generate descriptions of various topics without having been explicitly trained on those specific topics.

2. **`AgentType.REACT_DOCSTORE`: ReActDocstoreAgent**
   - This agent interacts with a document store. It's likely designed to handle tasks related to searching, retrieving, and interacting with a database of documents.

3. **`AgentType.SELF_ASK_WITH_SEARCH`: SelfAskWithSearchAgent**
   - This agent seems to be capable of asking questions to itself and then performing a search operation to find relevant information to answer those questions.

4. **`AgentType.CONVERSATIONAL_REACT_DESCRIPTION`: ConversationalAgent**
   - This agent is designed to engage in conversations. It's capable of generating responses in a conversational manner, potentially based on provided prompts or user inputs.

5. **`AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION`: ChatAgent**
   - This agent is tailored for chat-based interactions using a zero-shot approach. It can generate responses in a chat-like format without specific training on the topics.

6. **`AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION`: ConversationalChatAgent**
   - This is a variant of the `ChatAgent` specialized for conversational interactions. It can engage in back-and-forth conversations with users, generating responses in a chat-like manner.

7. **`AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION`: StructuredChatAgent**
   - This agent is designed for structured chat-based interactions using a zero-shot approach. It likely follows a specific format or structure in its responses.

8. **`AgentType.OPENAI_FUNCTIONS`: OpenAIFunctionsAgent**
   - This agent might be designed to utilize functions or capabilities provided by the OpenAI platform. It could perform various tasks or computations using OpenAI's functionalities.

9. **`AgentType.OPENAI_MULTI_FUNCTIONS`: OpenAIMultiFunctionsAgent**
   - Similar to the `OpenAIFunctionsAgent`, this agent may interact with multiple functions or capabilities provided by the OpenAI platform. It's likely capable of more complex operations or interactions.

These agents represent a range of capabilities, from generating descriptions to interacting with documents, performing searches, engaging in conversations, and utilizing functionalities provided by OpenAI. Each type is specialized for different tasks or scenarios within a broader system.

</details>


## PSE & Wiki 

In [12]:
from langchain import OpenAI
from langchain.agents import AgentType, Tool, initialize_agent, load_tools
from langchain.llms.openai import OpenAI
from langchain.tools import Tool, WikipediaQueryRun
from langchain.utilities import (
    GoogleSearchAPIWrapper,
    WikipediaAPIWrapper,

)


def pse(query):
    search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)
    return search.results(query, 5)

wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())


tools = [
    # Tool(
    #     name="Intermediate Answer",
    #     description="use this tool first. SEARCH IN GERMAN. Find offical german documents and laws concerning the users problem.\
    #         Retrieve sources and links to the document and give a short summary of its content.\
    #         Find official documents of Germany, preferably in pdf fromat",
    #     func=pse,
    # ),

    Tool(
        name="Intermediate Answer",
        description="use this tool when the first failed. \
            Find official documents of Germany, preferably in pdf fromat\
            Look in the footnotes and links of wikipedia pages for links to pdf's and sources\
            follow the links and find the source where the file can be downloaded",
        func=wiki.run,
    ),
]

agent = initialize_agent(
    tools,
    OpenAI(temperature=0),
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
)


agent.run(
    "What law regulates the duties of a landlord in berlin?"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: What is the name of the law that regulates the duties of a landlord in Berlin?
Intermediate answer: Page: Law of Germany
Summary: The law of Germany (German: das Recht Deutschlands), that being the modern German legal system (German: Deutsches Rechtssystem), is a system of civil law which is founded on the principles laid out by the Basic Law for the Federal Republic of Germany, though many of the most important laws, for example most regulations of the civil code (Bürgerliches Gesetzbuch, or BGB) were developed prior to the 1949 constitution.  It is composed of public law (öffentliches Recht), which regulates the relations between a citizen/person and the state (including criminal law) or two bodies of the state,  and the private law, (Privatrecht) which regulates the relations between two people or companies. It has been subject to a wide array of influences from Roman law, such as the Corpus Juris Civilis, to Napoleonic law, s

'The law of Germany (German: das Recht Deutschlands).'

# Serper, serapi, google-search, searchNG

In [79]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)


standard_tools = load_tools(["google-serper", "serpapi", "google-search", "searx-search"], searx_host="http://localhost:8888", llm=llm)

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run("Where can I find the law regulating the duties of a landlord in berlin germany?")



> Entering new AgentExecutor chain...
 I should search for the law online
Action: google_search
Action Input: "landlord duties berlin germany"
Observation: Full text in format: HTML PDF · text in German ; Section 239Surety ; Section 240Duty to supplement security ; Book 2. Law of obligations ; Division 1. Subject ... Legal obligations of secrecy, especially tax secrecy pursuant to Section 30 of the German Fiscal Code, as well as professional or special official secrecy ... 1 Act of 30.6.1959 101-3; Entry into force in Berlin on 10.8.1951 pursuant ... Where these duties are performed by an expert, the provisions of the General ...
Thought: I should look for a specific law
Action: google_search
Action Input: "landlord duties berlin germany law"
Observation: Full text in format: HTML PDF · text in German ; Section 239Surety ; Section 240Duty to supplement security ; Book 2. Law of obligations ; Division 1. Subject ... Book 10 of the German Code of Civil Procedure is largely based on the

'The law regulating the duties of a landlord in Berlin, Germany is found in Book 2 of the German Code of Civil Procedure, Section 239Surety, Section 240Duty to supplement security, and Book 10 of the German Code of Civil Procedure. Additionally, the German Civil Code and the Act of 30.6.1959 101-3 are also relevant.'

# Langchain + Streamlit Searchbar

In [ ]:
import streamlit as st

from langchain.agents import initialize_agent, AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun

with st.sidebar:
    openai_api_key = st.text_input(
        "OpenAI API Key", key="langchain_search_api_key_openai", type="password"
    )

st.title("🔎 LangChain - Chat with search")

"""
In this example, we're using `StreamlitCallbackHandler` to display the thoughts and actions of an agent in an interactive Streamlit app.
Try more LangChain 🤝 Streamlit Agent examples at [github.com/langchain-ai/streamlit-agent](https://github.com/langchain-ai/streamlit-agent).
"""

if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {
            "role": "assistant",
            "content": "Hi, I'm a chatbot who can search the web. How can I help you?",
        }
    ]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input(placeholder="Who won the Women's U.S. Open in 2018?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)

    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo", openai_api_key=openai_api_key, streaming=True
    )
    search = DuckDuckGoSearchRun(name="Search")
    search_agent = initialize_agent(
        [search],
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
    )
    with st.chat_message("assistant"):
        st_cb = StreamlitCallbackHandler(st.container(), expand_new_thoughts=False)
        response = search_agent.run(st.session_state.messages, callbacks=[st_cb])
        st.session_state.messages.append({"role": "assistant", "content": response})
        st.write(response)

# Chain to solve math problems

In [ ]:
from langchain.chains import PALChain
palchain = PALChain.from_math_prompt(llm=llm, verbose=True)
palchain.run("If my age is half of my dad's age and he is going to be 60 next year, what is my current age?")




# OUTPUT
# > Entering new PALChain chain...
# def solution():
#    """If my age is half of my dad's age and he is going to be 60 next year, what is my current age?"""
#    dad_age_next_year = 60
#    dad_age_now = dad_age_next_year - 1
#    my_age_now = dad_age_now / 2
#    result = my_age_now
#    return result
#
# > Finished chain.
# '29.5'

# You can check the default prompt
print(palchain.prompt.template)